In [1]:
import os
import glob

import pandas as pd
from pyteomics import parser, fasta

In [2]:
missing = pd.read_excel('../data/processed/missing_to_ensembl_20220411.xlsx')

In [3]:
hek = pd.read_csv('../data/processed/hek293t_va_20220525.csv')

In [4]:
missing

,uniprot_entry,uniprot_entry_name,ensembl_gene_id,ensembl_gene_name,ensembl_trs_id,ensembl_trs_name,length_bp,ensembl_protein_id,uniprot_isoform,length_aa
0,A0A075B6N3,TVBX1_HUMAN,ENSG00000211750,TRBV24-1,ENST00000390397,TRBV24-1-201,381,ENSP00000374920,A0A075B6N3,115
1,A0A075B6N4,TVBY1_HUMAN,ENSG00000282499,TRBV25-1,ENST00000390398,TRBV25-1-202,381,ENSP00000374921,A0A075B6N4,114
2,A0A075B6T7,TVA6_HUMAN,ENSG00000211780,TRAV6,ENST00000390428,TRAV6-201,404,ENSP00000438290,A0A075B6T7,132
3,A0A075B6U4,TVA7_HUMAN,ENSG00000211781,TRAV7,ENST00000390429,TRAV7-201,337,ENSP00000443297,A0A075B6U4,112
4,A0A075B6V5,TVA36_HUMAN,ENSG00000211815,TRAV36DV7,ENST00000390463,TRAV36DV7-201,356,ENSP00000450804,A0A075B6V5,113
...,...,...,...,...,...,...,...,...,...,...
1519,Q9Y5P0,O51B4_HUMAN,ENSG00000183251,OR51B4,ENST00000380224,OR51B4-201,933,ENSP00000369573,Q9Y5P0,310
1520,Q9Y6U7,RN215_HUMAN,ENSG00000099999,RNF215,ENST00000382363,RNF215-203,2011,ENSP00000371800,Q9Y6U7,377
1521,S4R3P1,HMN13_HUMAN,ENSG00000270394,MTRNR2L13,ENST00000604093,MTRNR2L13-201,1445,ENSP00000474570,S4R3P1,24
1522,S4R3Y5,HMN11_HUMAN,ENSG00000270188,MTRNR2L11,ENST00000604646,MTRNR2L11-201,1552,ENSP00000474861,S4R3Y5,24


In [11]:
missing.columns

Index(['uniprot_entry', 'uniprot_entry_name', 'ensembl_gene_id',
       'ensembl_gene_name', 'ensembl_trs_id', 'ensembl_trs_name', 'length_bp',
       'ensembl_protein_id', 'uniprot_isoform', 'length_aa', 'a549_tpm',
       'caco2_tpm', 'hacat_tpm', 'hek293t_tpm', 'hela_tpm', 'hepg2_tpm',
       'huh7_tpm', 'mcf7_tpm', 'saos2_tpm', 'skbr3_tpm', 'u2os_tpm'],
      dtype='object')

In [5]:
entries = missing[(missing['length_aa'] >= 90) & (missing.loc[:, ['hacat_tpm', 'hek293t_tpm', 'hela_tpm', 'hepg2_tpm']] >= 4).any(1)]['ensembl_gene_name']

KeyError: "None of [Index(['hacat_tpm', 'hek293t_tpm', 'hela_tpm', 'hepg2_tpm'], dtype='object')] are in the [columns]"

In [13]:
missing2 = missing[missing['ensembl_gene_name'].isin(entries)]

In [14]:
missing2 = missing2[missing2.columns.to_list()[:10] + ['hacat_tpm', 'hek293t_tpm', 'hela_tpm', 'hepg2_tpm']]

In [15]:
missing2.to_excel('../data/processed/missing_ensembl_select.xlsx', index=False)

In [16]:
missing2 = missing2[missing2['ensembl_gene_name'].isin(['CNPY1', 'MOSMO', 'TLCD5', 'CYB5RL', 'RNF215', 'C1orf154', 'TSPAN17', 'TMEM81', 'PPM1N'])]

In [18]:
missing2_hek = missing2[missing2.columns.to_list()[:10] + ['hek293t_tpm']]

In [6]:
missing2 = pd.read_excel('../reports/missing_hek_select_20220518.xlsx', index_col=[0,1,2,3]).reset_index()

In [7]:
missing2

,ensembl_gene_name,uniprot_isoform,protein_length_aa,ensembl_trs_id,ensembl_is_canonical,trs_length_bp,hek293t_tpm
0,AGAP9,Q5VTM2-2,658,ENST00000452145,True,2387,17.906937
1,ASAH2B,P0C7U1-1,165,ENST00000643851,False,5005,1.375991
2,ASAH2B,P0C7U1-1,165,ENST00000374006,False,632,3.907073
3,ASAH2B,P0C7U1-2,160,ENST00000374007,False,3715,3.989175
4,ASAH2B,P0C7U1-2,160,ENST00000647317,True,5145,2.919516
5,C1orf54,Q8WWF1-1,131,ENST00000369099,True,509,9.431817
6,C1orf54,Q8WWF1-1,131,ENST00000369102,False,1251,0.369875
7,CNPY1,Q3B7I2-1,92,ENST00000321736,False,2378,8.023424
8,CNPY1,Q3B7I2-1,92,ENST00000406197,False,560,19.871630
9,CYB5RL,Q6IPT4-4,247,ENST00000287899,False,5777,5.427784


In [36]:
with open('../data/raw/uniprot_human_proteome_canonical_plus_spliceforms_20220517.fasta') as f:
    proteome = f.read()
    proteome3 = fasta.FASTA(f)

In [50]:
proteome4 = pd.Series(proteome.split('>sp|'))

In [52]:
proteome4 = proteome4[1:].str.rstrip('>sp|')

In [53]:
proteome4 = proteome4.str.replace('\n', ' F=', n=1).str.replace('\n', '')

In [56]:
proteome4.iloc[0]

'A0A024RBG1|NUD4B_HUMAN Diphosphoinositol polyphosphate phosphohydrolase NUDT4B OS=Homo sapiens OX=9606 GN=NUDT4B PE=3 SV=1 F=MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSVR'

In [58]:
proteome5 = proteome4.str.split('\ F=').str[1]

In [59]:
proteome5.iloc[0]

'MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSVR'

In [28]:
proteome2 = proteome2[proteome2.str.contains('|'.join(missing2['uniprot_entry'].to_list()))].reset_index(drop=True).str.split('\ F=', expand=True)

In [29]:
proteome2.columns = ['entry', 'fasta']

In [30]:
def cleave(a):
    return list(parser.cleave(a, 'trypsin', min_length=7, max_length=25, missed_cleavages=1))

proteome2['peptides'] = proteome2['fasta'].apply(cleave)

In [31]:
proteome2['entry'] = proteome2['entry'].str.split('\|').str[0]

In [64]:
proteome2[]

,entry,fasta,peptides
0,Q96FV3,MPGKHQHFQEPEVGCCGKYFLFGFNIVFWVLGALFLAIGLWAWGEK...,DWIRDQLNLFINNNVK
0,Q96FV3,MPGKHQHFQEPEVGCCGKYFLFGFNIVFWVLGALFLAIGLWAWGEK...,LELEQQGFIHTK
0,Q96FV3,MPGKHQHFQEPEVGCCGKYFLFGFNIVFWVLGALFLAIGLWAWGEK...,MPGKHQHFQEPEVGCCGK
0,Q96FV3,MPGKHQHFQEPEVGCCGKYFLFGFNIVFWVLGALFLAIGLWAWGEK...,AYRDDIDLQNLIDFAQEYWSCCGAR
0,Q96FV3,MPGKHQHFQEPEVGCCGKYFLFGFNIVFWVLGALFLAIGLWAWGEK...,DDIDLQNLIDFAQEYWSCCGAR
...,...,...,...
18,Q8N819-3,MTCILVCFPGAPRPSEEAIRRELALDAALGCRIAELCASAQKPPSL...,RELALDAALGCR
18,Q8N819-3,MTCILVCFPGAPRPSEEAIRRELALDAALGCRIAELCASAQKPPSL...,ELALDAALGCR
18,Q8N819-3,MTCILVCFPGAPRPSEEAIRRELALDAALGCRIAELCASAQKPPSL...,SNPTHLGSALDMEA
18,Q8N819-3,MTCILVCFPGAPRPSEEAIRRELALDAALGCRIAELCASAQKPPSL...,IAELCASAQKPPSLNTVFR


In [33]:
proteome2 = proteome2.explode('peptides')

In [66]:
matches = []
for i in proteome2['peptides'].to_list():
    if isinstance(i, str):
        matches.append(len(proteome5[proteome5.str.contains(i)]))
    else:
        matches.append(0)

In [68]:
proteome2['matches'] = matches

In [70]:
proteome2 = proteome2.sort_values('entry')

In [77]:
proteome2['entry_base'] = proteome2['entry'].map(lambda x: x[:x.index('-')] if '-' in x else x)

In [85]:
proteome2['group_size'] = proteome2['entry'].map(proteome2.groupby('entry_base')['entry'].nunique())

In [145]:
proteome5[proteome5.str.contains('IQGRSLR')]

40565    MEDAHCTWLSLPGLPPGWALFAVLDGHGGARAARFGARHLPGHVLQ...
dtype: object

In [103]:
matches_here = []

for i,j in proteome2.iterrows():
    matches_here.append(len(pd.Series(proteome2[proteome2['entry_base']==j['entry_base']]['fasta'].unique()).str.contains(j['fasta'])))

In [102]:
pd.Series(proteome2[proteome2['entry_base']=='Q6IPT4']['fasta'].unique())

0    MMAEREEDDDTEEAWMQLRPTEPLPSQCCGSGCSPCVFDLYHRDLA...
1    MGLMSRYVESWRVGDTAFWRGPFGDFFYKPNQYGELLLLAAGTGLA...
2    MMAEREEDDDTEEAWMQLRPTEPLPSQCCGSGCSPCVFDLYHRDLA...
dtype: object

In [105]:
proteome2['matches_here'] = matches_here

In [107]:
proteome2 = proteome2.drop('group_size', axis=1)

In [109]:
proteome2['matches_other'] = proteome2['matches'] - proteome2['matches_here']

In [134]:
peptides_good = proteome2[proteome2.matches_other < 1]

In [115]:
missing2_hek

,uniprot_entry,uniprot_entry_name,ensembl_gene_id,ensembl_gene_name,ensembl_trs_id,ensembl_trs_name,length_bp,ensembl_protein_id,uniprot_isoform,length_aa,hek293t_tpm
643,Q3B7I2,CNPY1_HUMAN,ENSG00000146910,CNPY1,ENST00000406197,CNPY1-202,560,ENSP00000384514,Q3B7I2,92,19.871630
644,Q3B7I2,CNPY1_HUMAN,ENSG00000146910,CNPY1,ENST00000321736,CNPY1-201,2378,ENSP00000317439,Q3B7I2,92,8.023424
800,Q6IPT4,NB5R5_HUMAN,ENSG00000215883,CYB5RL,ENST00000287899,CYB5RL-201,5777,ENSP00000287899,Q6IPT4-4,247,5.427784
801,Q6IPT4,NB5R5_HUMAN,ENSG00000215883,CYB5RL,ENST00000534324,CYB5RL-209,6193,ENSP00000434343,Q6IPT4-1,315,0.508368
810,Q6P7N7,TMM81_HUMAN,ENSG00000174529,TMEM81,ENST00000367167,TMEM81-201,1332,ENSP00000356135,Q6P7N7,255,7.748528
841,Q6ZRR5,TLCD5_HUMAN,ENSG00000181264,TLCD5,ENST00000375095,TLCD5-202,3980,ENSP00000364236,Q6ZRR5-1,245,6.747156
842,Q6ZRR5,TLCD5_HUMAN,ENSG00000181264,TLCD5,ENST00000314475,TLCD5-201,1434,ENSP00000312672,Q6ZRR5-3,267,3.651433
843,Q6ZRR5,TLCD5_HUMAN,ENSG00000181264,TLCD5,ENST00000529187,TLCD5-204,3930,ENSP00000434862,Q6ZRR5-4,148,0.891785
969,Q8N819,PPM1N_HUMAN,ENSG00000213889,PPM1N,ENST00000401705,PPM1N-205,771,ENSP00000384318,Q8N819-3,112,1.719413
970,Q8N819,PPM1N_HUMAN,ENSG00000213889,PPM1N,ENST00000396737,PPM1N-203,872,ENSP00000379963,Q8N819-3,112,8.652978


In [141]:
peptides_good = peptides_good[~peptides_good['peptides'].isna()]

In [142]:
peptides_test = peptides_good['peptides'].to_list()

In [144]:
with open('peptides_for_unique.txt', 'w') as f:
    f.write('\n'.join(peptides_test))

In [143]:
peptides_test

['GDKIYQEFK',
 'LYFYSDAYRPLK',
 'MNDYKLEEDPVTK',
 'LCSEKSDLCETSANHTEL',
 'IYQEFKK',
 'KLYFYSDAYRPLK',
 'LEEDPVTKER',
 'LEEDPVTK',
 'SDLCETSANHTEL',
 'GIVDDLEIQR',
 'CLLCAGLTEDSYFLF',
 'LNPETFVAFCIIAMDRLTK',
 'AYTPISPANAEGYFEVLIK',
 'ELVSCCRR',
 'YVESWRVGDTAFWR',
 'FALPGNSQLGLRPGQHLILR',
 'DIARCLLCAGLTEDSYFLF',
 'LTKDTYR',
 'GPESQSCPSK',
 'VGDTAFWR',
 'WEAAQASKDR',
 'THFGHLGQDLIKELVSCCR',
 'TFFVLSQESSSEQLPWSYQEK',
 'LNPETFVAFCIIAMDR',
 'DLARWEAAQASK',
 'SLLRGPESQSCPSK',
 'CYQMGLMSRYVESWR',
 'TFESIYLK',
 'RKPFALVCGSAEFTK',
 'THFGHLGQDLIK',
 'TFESIYLKTFLQEQAR',
 'VRFALPGNSQLGLRPGQHLILR',
 'TFLQEQAR',
 'TFLQEQARFWNVR',
 'KPFALVCGSAEFTK',
 'KPFALVCGSAEFTKDIAR',
 'CYQMGLMSR',
 'ELVSCCR',
 'WEAAQASK',
 'TFLQEQARFWNVR',
 'DIARCLLCAGLTEDSYFLF',
 'YVESWRVGDTAFWR',
 'ELVSCCRR',
 'TFFVLSQESSSEQLPWSYQEK',
 'RKPFALVCGSAEFTK',
 'THFGHLGQDLIKELVSCCR',
 'MGLMSRYVESWR',
 'TFLQEQAR',
 'CLLCAGLTEDSYFLF',
 'ELVSCCR',
 'THFGHLGQDLIK',
 'KPFALVCGSAEFTK',
 'TFESIYLK',
 'KPFALVCGSAEFTKDIAR',
 'VGDTAFWR',
 'TFE

In [131]:
proteome5[proteome5.str.contains('ELVSCCR')]

15921    MMAEREEDDDTEEAWMQLRPTEPLPSQCCGSGCSPCVFDLYHRDLA...
39725    MGLMSRYVESWRVGDTAFWRGPFGDFFYKPNQYGELLLLAAGTGLA...
39726    MMAEREEDDDTEEAWMQLRPTEPLPSQCCGSGCSPCVFDLYHRDLA...
dtype: object

In [133]:
proteome5.apply(len).mean()

575.2809880274872

In [147]:
proteome2[['entry', 'peptides']].to_csv('missing_select_peptides_20220518.txt', sep='\t', index=False)